In [7]:
import pandapower as pp
# 1. 创建一个空的pandapower网络
net = pp.create_empty_network()

# 2. 添加4个母线
# 母线电压通常设定为 kV
bus1 = pp.create_bus(net, vn_kv=20., name="Bus 1")
bus2 = pp.create_bus(net, vn_kv=20., name="Bus 2")
bus3 = pp.create_bus(net, vn_kv=20., name="Bus 3")
bus4 = pp.create_bus(net, vn_kv=20., name="Bus 4")
bus5 = pp.create_bus(net, vn_kv=20., name="Bus 4")

# 3. 母线 1、2 通过 switch 相连
# 修正后的创建方式：bus=一个母线, element=另一个母线, et="b"
pp.create_switch(net, bus=bus1, element=bus2, et="b", closed=True, name="Switch 1-2")

# 4. 母线 3、4 通过 switch 相连
# 修正后的创建方式：bus=一个母线, element=另一个母线, et="b"
# pp.create_switch(net, bus=bus3, element=bus4, et="b", closed=True, name="Switch 3-4")

# 5. 线路 1 连接母线 1、3
# 需要定义线路参数：长度 km, r_ohm_per_km, x_ohm_per_km, c_nf_per_km, max_i_ka
# 这里我们使用一些示例值
line1 = pp.create_line(net, from_bus=bus1, to_bus=bus5, length_km=5.0,
               std_type="NAYY 4x50 SE", name="Line 1-3") # 使用标准线路类型，pandapower会填充参数

## 在线路 1 和母线 3、4 之间添加两个开关
pp.create_switch(net, bus=bus3, element=bus5, et="b", closed=False, name="Switch 3-1")
pp.create_switch(net, bus=bus4, element=bus5, et="b", closed=True, name="Switch 4-1")
# 6. 母线 1 有一个发电机 (可以是一个发电机 gen 或外部网架 ext_grid)
# 通常用 ext_grid 来模拟一个连接到无限大母线或强电网的接入点
# 如果是发电机，需要设置 P_mw 和 vm_pu (电压标幺值)
pp.create_ext_grid(net, bus=bus1, vm_pu=1.02, va_degree=0, name="External Grid at Bus 1")

# 如果您想模拟一个普通的同步发电机 (gen)，示例如下：
# pp.create_gen(net, bus=bus1, p_mw=10., vm_pu=1.01, name="Generator at Bus 1")


# 7. 母线 3 有一个 PQ 负荷
# 需要定义 P_mw 和 Q_mvar
pp.create_load(net, bus=bus3, p_mw=5., q_mvar=2., name="Load at Bus 3")

# 打印网络概览以确认所有组件是否正确添加
print(net)

# 运行潮流计算（可选，但通常是创建算例的目的）
try:
    pp.runpp(net)
    print("\n潮流计算结果：")
    print("母线电压 (pu):\n", net.res_bus.vm_pu)
    print("线路潮流 (MW):\n", net.res_line.p_from_mw)
except Exception as e:
    print(f"\n潮流计算失败: {e}")
    print("请检查网络连接或参数设置。")


numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)


This pandapower network includes the following parameter tables:
   - bus (5 elements)
   - load (1 element)
   - switch (3 elements)
   - ext_grid (1 element)
   - line (1 element)

潮流计算结果：
母线电压 (pu):
 0    1.02000
1    1.02000
2        NaN
3    1.02007
4    1.02007
Name: vm_pu, dtype: float64
线路潮流 (MW):
 0    0.000036
Name: p_from_mw, dtype: float64
